# Imports and initialization of general parameters
***

In [2]:
from helpers.pareto_fairness import compute_pareto_metrics
from config.info import AGES, RACES, GENDERS, COMBS_BASELINE
from visualization.subgroup_distribution import plot_dist
from dataprocess.dataloader import load_data
from dataprocess.dataclass import Data
from config.get_args import get_args
from sklearn.metrics import mean_squared_error
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from lightning import seed_everything
from plotly.subplots import make_subplots
import numpy as np
import argparse
import pandas as pd
import random
import torch.nn as nn
import pickle

loss_fct = nn.CrossEntropyLoss()


# Auto reload part
%load_ext autoreload
%autoreload 2

# Load the data and the response
***